# Gender Wage Gap Analysis: Comprehensive Feature Set

This notebook performs a Double Machine Learning (DML) analysis of the gender wage gap using **all available variables** in the dataset. 

## Methodology
1.  **Dynamic Data Cleaning**: Automatically remove low-quality columns (high missingness, zero variance).
2.  **Redundancy Removal**: Identify and drop highly correlated features to prevent multicollinearity.
3.  **Feature Engineering**: Create essential economic variables (Experience, Log Wage) while preserving the richness of the raw data.
4.  **Double Machine Learning**: Estimate the causal effect of gender on wages controlling for the full, high-dimensional feature set.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import doubleml as dml
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully.")

In [ ]:
# Load Data
try:
    df = pd.read_excel('../data/raw/sample of sample ACS.xlsx')
    print(f"Data loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print("ERROR: Data file not found. Please ensure 'sample of sample ACS.xlsx' is in '../data/raw/'.")
    # Create dummy data for demonstration if file is missing (so notebook can run)
    print("Creating dummy data for demonstration...")
    df = pd.DataFrame({
        'INCWAGE': np.random.lognormal(10, 1, 1000),
        'SEX': np.random.choice(['Male', 'Female'], 1000),
        'AGE': np.random.randint(18, 65, 1000),
        'EDUC': np.random.choice(['HS', 'College', 'Grad'], 1000),
        'WKSWORK1': np.random.randint(40, 52, 1000),
        'UHRSWORK': np.random.randint(30, 60, 1000),
        'EMPSTAT': ['Employed'] * 1000
    })

In [ ]:
# Initial Filtering
# We only want employed individuals with positive wages
print("Filtering for employed workers with positive wages...")
original_len = len(df)
df = df[df['INCWAGE'] > 0]
df = df[df['EMPSTAT'] == 'Employed']
df = df[df['WKSWORK1'] > 0]
df = df[df['UHRSWORK'] > 0]
df = df[(df['AGE'] >= 18) & (df['AGE'] <= 65)]
print(f"Rows remaining: {len(df)} (Dropped {original_len - len(df)} rows)")

## Feature Engineering: Core Variables

We create the fundamental variables required for the wage gap analysis:
*   **`LOG_WAGE`**: The natural logarithm of annual wage income. This is standard in labor economics because wages are right-skewed and the log-linear model allows coefficients to be interpreted as percentage changes.
*   **`FEMALE`**: A binary indicator (1 if Female, 0 if Male). This is our **treatment variable**.


In [ ]:
# Create Target and Treatment
df['LOG_WAGE'] = np.log(df['INCWAGE'])
df['FEMALE'] = (df['SEX'] == 'Female').astype(int)

print("Created LOG_WAGE and FEMALE variables.")

## Feature Engineering: Experience

Since actual work experience is not in the dataset, we calculate **Potential Experience** using the Mincer equation approach:
$$ \text{Potential Experience} = \text{Age} - \text{Years of Education} - 6 $$

We also include squared terms (`AGE_SQ`, `POTENTIAL_EXP_SQ`) to capture the non-linear relationship between age/experience and earnings (wages typically rise early in a career and flatten out).


In [ ]:
# Create Experience Variables
df['AGE_SQ'] = df['AGE'] ** 2

# Extract years of education (simplified mapping)
# Assuming EDUCD or EDUC exists. If not, we skip.
if 'EDUCD' in df.columns:
    # Map EDUCD to approximate years (simplified)
    df['EDUC_YEARS'] = pd.to_numeric(df['EDUCD'].astype(str).str[:1], errors='coerce').fillna(12)
else:
    df['EDUC_YEARS'] = 12 # Default

df['POTENTIAL_EXP'] = df['AGE'] - df['EDUC_YEARS'] - 6
df['POTENTIAL_EXP'] = df['POTENTIAL_EXP'].clip(lower=0) # No negative experience
df['POTENTIAL_EXP_SQ'] = df['POTENTIAL_EXP'] ** 2

print("Created Experience variables.")

## Dynamic Data Cleaning

We now process **all other variables** in the dataset. To ensure the model is robust, we apply the following filters:

1.  **High Missingness**: Drop columns with > 50% missing values.
2.  **Zero Variance**: Drop columns that have only 1 unique value (they provide no information).
3.  **Object Conversion**: Convert all categorical (object) columns to numeric using Label Encoding.


In [ ]:
# 1. Drop High Missingness
missing_threshold = 0.5
missing_ratio = df.isnull().mean()
drop_missing = missing_ratio[missing_ratio > missing_threshold].index.tolist()
df = df.drop(columns=drop_missing)
print(f"Dropped columns with > {missing_threshold*100}% missing values: {drop_missing}")

# 2. Drop Zero Variance (Single Value)
nunique = df.nunique()
drop_single = nunique[nunique <= 1].index.tolist()
df = df.drop(columns=drop_single)
print(f"Dropped {len(drop_single)} columns with only 1 unique value.")

# 3. Handle Remaining Missing Values
# Fill numeric with 0, object with 'Unknown'
num_cols = df.select_dtypes(include=[np.number]).columns
obj_cols = df.select_dtypes(include=['object']).columns

df[num_cols] = df[num_cols].fillna(0)
df[obj_cols] = df[obj_cols].fillna('Unknown')

# 4. Label Encode Categorical Variables
le = LabelEncoder()
for col in obj_cols:
    df[col] = df[col].astype(str)
    df[col] = le.fit_transform(df[col])
    
print(f"Processed {len(obj_cols)} categorical columns.")
print(f"Final Data Shape: {df.shape}")

## Redundancy Removal

We calculate the correlation matrix for all features. If two features have a correlation greater than **0.95**, they are effectively redundant. We drop one of them to reduce dimensionality and improve model stability.


In [ ]:
# Calculate Correlation Matrix
# (Using a subset or efficient method if data is huge, but here we do full)
print("Calculating correlations...")
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Keep essential variables even if correlated
essentials = ['LOG_WAGE', 'FEMALE', 'AGE', 'AGE_SQ', 'POTENTIAL_EXP', 'POTENTIAL_EXP_SQ']
to_drop = [col for col in to_drop if col not in essentials]

df = df.drop(columns=to_drop)
print(f"Dropped {len(to_drop)} highly correlated features (>0.95).")
print(f"Remaining columns: {len(df.columns)}")

## Variable Explanation

The following table lists the variables used in the analysis after cleaning and redundancy removal.


In [ ]:
# Variable Explanation
feature_info = []
for col in df.columns:
    if col not in ['LOG_WAGE', 'FEMALE', 'INCWAGE', 'SEX']:
        dtype = str(df[col].dtype)
        feature_info.append({'Feature': col, 'Type': dtype})

feature_df = pd.DataFrame(feature_info)
from IPython.display import display, Markdown
print(f"Total Features Used: {len(feature_df)}")
display(feature_df)


## Double Machine Learning Analysis

We now run the DoubleML Partial Linear Regression (`DoubleMLPLR`) model.

*   **Outcome ($Y$)**: `LOG_WAGE`
*   **Treatment ($D$)**: `FEMALE`
*   **Controls ($X$)**: All other remaining variables.

We use **LightGBM** as the machine learning learner for both the nuisance functions (predicting wage and predicting gender). LightGBM is chosen for its speed and ability to handle large, high-dimensional datasets.


In [ ]:
# Define Features
y_col = 'LOG_WAGE'
d_col = 'FEMALE'
x_cols = [c for c in df.columns if c not in [y_col, d_col, 'INCWAGE', 'SEX']] # Exclude raw target/treatment

print(f"Using {len(x_cols)} control variables.")

# Initialize DoubleML Data
dml_data = dml.DoubleMLData(df,
                           y_col=y_col,
                           d_cols=d_col,
                           x_cols=x_cols)

# Initialize Learners
# We use LightGBM for speed and performance
ml_l = LGBMRegressor(n_estimators=100, learning_rate=0.1, n_jobs=-1, verbose=-1)
ml_m = LGBMClassifier(n_estimators=100, learning_rate=0.1, n_jobs=-1, verbose=-1)

# Initialize DoubleML Model
np.random.seed(42)
dml_plr = dml.DoubleMLPLR(dml_data,
                          ml_l=ml_l,
                          ml_m=ml_m,
                          n_folds=3)

# Fit Model
print("Fitting DoubleML model (this may take a moment)...")
dml_plr.fit()

# Display Results
print("\n" + "="*50)
print("DOUBLE ML RESULTS (ALL FEATURES)")
print("="*50)
print(dml_plr.summary)

gap_pct = (np.exp(dml_plr.coef[0]) - 1) * 100
print(f"\nEstimated Gender Wage Gap: {gap_pct:.2f}%")
print(f"Women earn {100 + gap_pct:.1f} cents for every dollar men earn (adjusted for all features).")